# Frontier Exploration Problem

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/CollaborativeRoboticsLab/foundations-of-robotics-labs/blob/master/4-algorithms/01-exploration-algorithm.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CollaborativeRoboticsLab/foundations-of-robotics-labs/master?filepath=4-algorithms/01-exploration-algorithm.ipynb)

In this notebook, you will review a frontier-based exploration algorithm. The algorithm can be tested in a simulated environment using the Gazebo simulator.

![Frontier Exploration](./docs/images/frontiers-rviz.png)

### What is a frontier?

A frontier is a line that separates explored and unexplored areas.

### What is a frontier-based exploration algorithm?

A frontier-based exploration algorithm is an algorithm that uses frontiers to explore an unknown environment. The algorithm works as follows:

1. Find a frontier.
2. Move the robot to the frontier.
3. Repeat.

![frontier exploration algorithm](./docs/images/frontier-explore-diagram.png)

#### How to find a frontier?

To find a frontier, the robot uses a grid based map representation of the world. Each cell in the grid is either occupied, free or unexplored. Frontiers are unexplored cells that are adjacent to explored cells. The algorithm finds frontiers by looking for unexplored cells that are *close by*. The algorithm then selects the closest frontier and moves the robot to it.

#### Find unexplored cells nearby

The robot posistion in the map is calculated and then this position can be compared with the position of unexplored cells. The distance between the robot and the unexplored cells is the [euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance). The `numpy` library can be used to calculate the distance between the robot and the unexplored cells. The equation is as follows:

$$ distance = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2} $$

Where $(x_1, y_1)$ is the unexplored cell position and $(x_2, y_2)$ is the robot position.

### Distance: A numpy code example

```python
# for example
import numpy as np

'''
example data
'''

robot_position = robot.translation # provided by a helper function that looks up the robot's position using transforms
frontier_cells = [{'x': 1.0, 'y': 2.0}, {'x': 3.0, 'y': 4.0}, ...] # provided by a helper function that finds the unexplored cells from the current world map

'''
calculate the distance between the robot and the frontier cells
'''

# the np.sqrt function calculates the square root
# use it to get the root of the sum of the squares of the differences (euclidean distance)
# here is an example of its usage
for cell in frontier_cells:
    distance = np.sqrt((cell['x'] - robot_position.x)**2 + (cell['y'] - robot_position.y)**2)
```

### Sorting the frontiers

The frontiers are sorted by distance. The closest frontier is selected and the robot moves to it. The robot then repeats the process. The sorting can be done using the `sorted` function in Python. The `sorted` function takes a list and a key. The key is a function that returns the value to be used for sorting. In this case, the key is the distance between the robot and the frontier.

```python
# for example
frontier_cells = [{'x': 1.0, 'y': 2.0}, {'x': 3.0, 'y': 4.0}, ...] # provided by a helper function that finds the unexplored cells from the current world map

# sort the frontiers by distance
sorted_frontiers = sorted(frontier_cells, key=lambda cell: np.sqrt((cell['x'] - robot_position.x)**2 + (cell['y'] - robot_position.y)**2))
```

### Pythonic List Comprehension

Python has a feature called list comprehension. List comprehension is a concise way to create lists. It is a powerful feature that can be used to create lists in a single line of code. Here is an example of list comprehension:

```python
# for example
frontier_cells = [{'x': 1.0, 'y': 2.0}, {'x': 3.0, 'y': 4.0}, ...] # provided by a helper function that finds the unexplored cells from the current world map

# calculate the distance between the robot and the frontier cells
distances = [np.sqrt((cell['x'] - robot_position.x)**2 + (cell['y'] - robot_position.y)**2) for cell in frontier_cells]
```

In the above example, the `distances` list is created using list comprehension. The list comprehension iterates over the `frontier_cells` list and calculates the distance between the robot and the frontier cells all in a single line of code!

---

### Test the code

Run the ROS launch file to start the Gazebo simulator and the RViz visualizer.

```bash
# in a terminal on the computer run the following command
roslaunch sphero_rvr_algorithms ball_maze.launch

# to view the result in RViz run the following command
roslaunch sphero_rvr_desktop display.launch
```

#### Make sure the environment is set up correctly

The following cell adds rospy to the python path. Run this cell before running the code.


In [ ]:
"""setting up ros enrionment variables"""

# this set of lines is to make sure that the python scripts can find the ROS libraries
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages')
sys.path.append('/usr/lib/python3/dist-packages')

### The code:

In [ ]:
"""
Frontier exploration assignment

Implement a frontier exploration algorithm
"""


import rospy
import actionlib
from sensor_msgs.msg import PointCloud2, PointField
from nav_msgs.msg import OccupancyGrid
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import numpy as np
import tf2_ros


class FrontierExplorer(object):
    def __init__(self, rate=1.0):
        """
        Class constructor
        This method should initialize class variables and set up the node
        the __init__ method is called when the object is created
        """
        # create an exploration timer to explore the environment every 1/rate seconds
        self.rate = rate
        self.exploration_timer = rospy.Timer(rospy.Duration(1.0 / self.rate), self.explore_timer_cb, oneshot=True)

        # subscribe to the occupancy grid
        self.occupancy_grid_sub = rospy.Subscriber('/map', OccupancyGrid, self.occupancy_grid_cb, queue_size=1)
        self.occupancy_grid = None

        # map meta data
        self.map_metadata = None

        # create a move base client to send goals to move_base
        self.move_base_client = actionlib.SimpleActionClient('/move_base', MoveBaseAction)
        self.move_base_client.wait_for_server()

        # create a publisher for the frontier cells as points
        self.frontier_pub = rospy.Publisher('~frontier_cells', PointCloud2, queue_size=1)
        self.frontier_cloud = PointCloud2()

        # create a tf buffer and listener
        self.tf_buffer = tf2_ros.Buffer()
        self.tf_listener = tf2_ros.TransformListener(self.tf_buffer)

        '''
        Any extra set up? put your code here
        '''

    def find_frontier(self, occupancy_grid):
        """
        find the frontier cells in the map

        Args:
            occupancy_grid (numpy.array): occupancy grid map of the environment

        Returns:
            (list): list containing the frontier cells in the map
            The list should contain dictionaries with keys 'x' and 'y' representing the x and y coordinates of the frontier cell
        """

        # empty frontier cells list
        frontier_cells = []

        # empty occ grid guard
        if occupancy_grid is None:
            rospy.logwarn("Occupancy grid is empty")
            return []

        # find cells that are not yet explored
        # cells in the occupancy grid with a value of -1 are not yet explored
        # these are the frontier cells

        # the occupancy grid is a 2D numpy array
        # you will need to loop through the occupancy grid values to find the frontier cells

        # loop through the occupancy grid
        # this is a 2D numpy array
        # it has a shape (height, width)
        # you can loop through the grid using two for loops
        # the first loop goes through the rows (height)
        # the second loop goes through the columns (width)
        # the value of the cell at the current row and column is given by occupancy_grid[i, j]

        # Get the indices of the frontier cells
        frontier_indices = np.where(occupancy_grid == -1)

        # Convert the indices to real-world coordinates
        x_values = frontier_indices[1] * self.map_metadata.resolution + self.map_metadata.origin.position.x
        y_values = frontier_indices[0] * self.map_metadata.resolution + self.map_metadata.origin.position.y

        # Create a boolean mask for the cells within the desired range
        mask = (np.abs(x_values) < 2.8) & (np.abs(y_values) < 2.8)

        # Apply the mask to the x and y values
        x_values = x_values[mask]
        y_values = y_values[mask]

        # Combine the x and y values into a list of dictionaries
        frontier_cells = [{'x': x, 'y': y} for x, y in zip(x_values, y_values)]

        # publish the frontier cells as a point cloud
        # this is for visualization
        # you can visualize the frontier cells in rviz
        # the point cloud should be published in the map frame of reference
        # you can use the self.frontier_pub publisher to publish the point cloud
        self.publish_frontier_cloud(frontier_cells)

        # return the frontier cells
        return frontier_cells

    def explore(self):
        """
        Implement a frontier exploration algorithm

        work out the frontier cells in the map
        then move to the closest frontier (challenge)
        """

        #############
        ### SENSE ###
        #############

        # here we will find the frontier cells in the map
        # as well as the robot's pose in the map frame of reference

        # create a copy of the occupancy grid
        occ = self.occupancy_grid.copy()

        # find the frontier cells
        frontier_cells = self.find_frontier(occ)

        # the selected frontier to move toward
        selected_frontier_cell = None

        # get the robot's pose in the map frame
        robot = self.get_robot_pose()

        robot_position = robot.translation

        '''
        (challenge)
        find the nearest frontier to the robot

        (hint)
        use the robot's pose in the 'map' frame of reference to find the nearest frontier
        use the euclidean distance to find the nearest frontier

        your code here
        '''

        #############
        ### THINK ###
        #############

        # remove cells that are too close to the robot
        # this will help the robot to explore new areas outside its current vicinity
        # at least min_distance meters away from the robot
        # (hint): use this snippet to understand how to calculate the distance between two points as a condition

        min_distance = 0.5
        frontier_cells = [cell for cell in frontier_cells if np.sqrt((cell['x'] - robot_position.x)**2 + (cell['y'] - robot_position.y)**2) > min_distance]

        # find frontier cells nearest to the robot
        # calculate the distance of each frontier cell from the robot
        # the distance is the euclidean distance between the robot and the frontier cell
        # sort the frontier cells by distance using the sorted function
        # the closest frontier cell is the first in the list
        # for example:
        # nearest_frontier = sorted_frontiers[0]

        sorted_frontiers = sorted(frontier_cells, key=lambda cell: np.sqrt((cell['x'] - robot_position.x)**2 + (cell['y'] - robot_position.y)**2)) # the sorted frontier cells
        nearest_frontier = sorted_frontiers[0]  # the nearest frontier cell

        # next we need to select the nearest frontier cell to move to
        # the selected frontier cell is the one the robot will move to
        # don't forget that the frontier positions are in dictionary format
        # for example:
        # selected_frontier_cell = (???, ???)  # replace ??? with the selected frontier cell x and y coordinates

        x = nearest_frontier['x']
        y = nearest_frontier['y']

        selected_frontier_cell = (x, y) # the selected frontier cell

        '''
        your code finishes here
        '''

        # no selected frontier cell guard
        if selected_frontier_cell is None:
            return

        print("selected frontier cell: ", selected_frontier_cell)

        ###########
        ### ACT ###
        ###########

        # all we need to do once a frontier cell is selected is to move to it
        # this method moves to the selected frontier cell
        self.move_base_to_frontier(selected_frontier_cell)

    # state timer callback
    def explore_timer_cb(self, event):
        '''
        the exploration timer callback function
        this method is called whenever the exploration timer is triggered
        it represents the whole 'sense-think-act' cycle
        '''
        print("exploring...")

        # do the exploration method
        self.explore()

        # restart the timer once the exploration is done
        self.exploration_timer = rospy.Timer(rospy.Duration(1.0 / self.rate), self.explore_timer_cb, oneshot=True)

    # occupancy grid callback
    def occupancy_grid_cb(self, msg):
        '''
        occupancy grid message callback function

        This method is called whenever a new occupancy grid message (a map) is received
        This method represents the part of the 'sense' step of the sense-think-act cycle
        '''
        # convert the occupancy grid message to a numpy array
        self.occupancy_grid = np.array(msg.data).reshape((msg.info.height, msg.info.width))
        # store the map metadata
        self.map_metadata = msg.info


    ### helper functions ###

    # move to selected frontier using move_base
    def move_base_to_frontier(self, coord):
        # create a move base goal message
        goal = MoveBaseGoal()
        goal.target_pose.header.frame_id = 'map'
        goal.target_pose.header.stamp = rospy.Time.now()

        # set the goal position
        goal.target_pose.pose.position.x = coord[0]
        goal.target_pose.pose.position.y = coord[1]
        goal.target_pose.pose.position.z = 0.0

        # set goal orientation (facing forward)
        goal.target_pose.pose.orientation.x = 0.0
        goal.target_pose.pose.orientation.y = 0.0
        goal.target_pose.pose.orientation.z = 0.0
        goal.target_pose.pose.orientation.w = 1.0

        # send the goal
        self.move_base_client.send_goal(goal)

        # wait for the goal to complete
        self.move_base_client.wait_for_result()

    # robot pose in map frame
    def get_robot_pose(self):
        '''
        This method returns the robot's pose in the map frame of reference
        It uses the tf buffer and listener to get the transform from the base_link to the map frame
        '''
        # find robot pose in the map frame
        try:
            # get the transform from the base_link to the map frame
            transform = self.tf_buffer.lookup_transform(
                'map', 'base_link', rospy.Time(0), rospy.Duration(1.0))
        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
            rospy.logwarn("Failed to get transform from base_link to map")
            return None

        # return the robot pose
        return transform.transform

    # publish frontier cells as a point cloud
    def publish_frontier_cloud(self, frontier_cells):
        self.frontier_cloud.header.frame_id = 'map'
        self.frontier_cloud.header.stamp = rospy.Time.now()
        self.frontier_cloud.width = len(frontier_cells)
        self.frontier_cloud.height = 1
        self.frontier_cloud.fields = [
            PointField('x', 0, PointField.FLOAT32, 1),
            PointField('y', 4, PointField.FLOAT32, 1),
            PointField('z', 8, PointField.FLOAT32, 1),
            PointField('intensity', 12, PointField.FLOAT32, 1)
        ]
        self.frontier_cloud.is_bigendian = False
        self.frontier_cloud.point_step = 16
        self.frontier_cloud.row_step = 16 * len(frontier_cells)
        self.frontier_cloud.is_dense = True
        self.frontier_cloud.data = np.array(
            [(cell['x'], cell['y'], 0, 0.5) for cell in frontier_cells],
            dtype=[
                ('x', np.float32),
                ('y', np.float32),
                ('z', np.float32),
                ('intensity', np.float32)
            ]
        ).tobytes()

        # publish the point cloud
        self.frontier_pub.publish(self.frontier_cloud)

# main function
def main():
    # initialize a node
    rospy.init_node('frontier_explorer')

    # create a frontier explorer object
    frontier_explorer = FrontierExplorer(0.5) # explore every 2 seconds

    # spin
    rospy.spin()

print("frontier_explorer class and main function defined")

---

### Main Function

**Run the following cell to test the code.**

> **Note:**
>
> **Don't forget to run the cell that defines the `FrontierExplorer` class before running the main function, and after any changes you make to the class.**
>
> **After running main, restart the kernel to run main again.**

In [ ]:
# run main
main()

### Testing & Debugging

If you are having trouble with your code, try to print out information to understand what is going wrong. Add `print()` statements to your code to see the values of variables and to understand the flow of your code.

#### ROS Network

Use the ROS Command Line Tools to check the ROS network. The following commands can be useful:

```bash
# List all active ROS nodes
rosnode list

# List all active ROS topics
rostopic list

# Get information about a specific topic
rostopic info /topic_name

# Get information about a specific node
rosnode info /node_name
```

### Troubleshooting

- Remember that the python functions that are needed have documenation online.
- If you are having trouble with the code, try to break it down into smaller parts and test each part individually.
- If you are having trouble with the robot, try to move it around using the teleop tool (diff drive marker) in RViz.